## A notebook to help you use the GLANCE method. 

In [1]:
#packages that we will use
using MAT
using MatrixNetworks
using ScikitLearn
include("includeall.jl")

  Building Conda ─→ `~/.julia/packages/Conda/3rPhK/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/zqDXB/deps/build.log`


PyObject <module 'numpy' from '/Users/hnassar/.julia/conda/3/lib/python3.7/site-packages/numpy/__init__.py'>

First, let's get some data. For this example, we will use the smallest dataset from the FB100 dataset, the Caltech36 dataset. In this data, we have attribute values in the vector `labels`. Here, `labels[i]` correspond to a unique number of the dorm where student `i` resided.

In [2]:
T = MAT.matopen("Caltech36.mat")
attributes = read(T,"local_info")
A = read(T,"A")
A,lccv = largest_component(A);
attributes = attributes[lccv,:];
clabels = attributes[:,5]
;

In [3]:
function GLANCE(A;TSfunction=x->x,fromk=3,tok=20,trialnb=50000,mydims = 25)
    cc = scomponents(A)
    @assert length(cc.sizes) == 1
    @assert issymmetric(A)
    tfn = TSNE(n_components=2)
    # step 1: Turan Shadow reweighting
    G = TuranShadow_matrix(A,TSfunction,fromk,tok,1,trialnb)
    # step 2: Embedding on the new weighted matrix
    x2,x3,X = x2_x3_from_spectral_embedding(G;tol=1e-12,maxiter=300,dense=96,nev=mydims,checksym=true)
    # step 3: transform via t-sne
    xy_coords = tfn.fit_transform(X)
    return xy_coords
end

GLANCE (generic function with 1 method)

In [4]:
xy = GLANCE(A);

size(X) = (762, 25)


In [5]:
include("vis_paper.jl")

regenerate_ppr_plots (generic function with 1 method)

In [6]:
generate_nice_plot(xy,A,"Glance_layout.png";labels=clabels,invpermval=true,mymarkeralpha=0.6,mymarkersize=2)

Compressing counts... 2545 / 16651
Compressing counts... 5044 / 16651
Compressing counts... 7498 / 16651
Compressing counts... 10148 / 16651
Compressing counts... 12498 / 16651
Compressing counts... 14920 / 16651
 15.132297 seconds (8.79 M allocations: 733.391 MiB, 3.70% gc time)


And the image will be saved in the same folder. Note that we use this method to save to a file directly because we produce a high resolution image and it may break your browser.